# IMPORTING LIBRARIES

In [1]:
import os
from pathlib import Path
import pywikibot
from OCR_worker.pdftotext import convert_pdf_to_text, extract_text # IMPORTING PYTHON SCRIPTs WHICH ADDS A TEXT LAYER
from IPython.display import clear_output

# DEFINING FUNCTIONS

In [16]:
def check_MIME_pdf(file_name):
    '''
    FOR HANDLING ERROR FOR FILENAMES LIKE  "OO_No._45_dt._09.05.2023"  
    GIVES ERROR >>> verification-error: File extension ".2023" does not match the detected MIME type of the file (application/pdf).
    '''
    _file_name = ''
    isChanged = False

    _file_name = file_name[:-4]  # EXCLUDING '.pdf' EXTENSION 
    _file_name = _file_name.replace('.', '')
    _file_name = _file_name+'.pdf'
    isChanged = _file_name != file_name

    return (_file_name, isChanged)


def upload_pdf_files(file_path, wiki_url, username, addSummary = False, summary=' '):
    '''
    UPLOADS ALL PDF FILES FROM A DIRECTORY
    TO MEDIAWIKI WITH AN OCR TEXT LAYER

    IF addSummary is True,
        if USER PROVIDES SOME SUMMARY THEN IT ADDS THAT IN THE COMMENT SECION,
        ELSE, IT SCRAPS THE TEXT AND ADDS THE TEXT IN THE COMMENT SECTION
    '''

    site = pywikibot.Site(user=username, code="en", fam="mediawiki")
    site.login()

    for file_name in os.listdir(file_path):
        if file_name.endswith('.pdf'):
            (new_file_name, isChanged) = check_MIME_pdf(file_name)
            
            print("processing ", {file_name})
            _file_path = file_path
            ffile_path = os.path.join(file_path, file_name)

            if isChanged:
                file_name = new_file_name

            output_file = "new/ocr_"+file_name     # SAVES FILE IS new DIRECTORY
            output_file_path = os.path.join(_file_path, output_file)
            
            convert_pdf_to_text(pdf_file=ffile_path, output_file=output_file_path)

            clear_output(wait = True)

            print("Text Layer Added")

            page_name = file_name[:-4]  # Remove the '.pdf' extension from the file name

            
            if addSummary:
                if len(summary) < 2:
                    textt = extract_text(output_file_path)
                    summary = " \n ".join([textt[page] for page in range(len(textt))])
                    
                print(summary)

            page = pywikibot.FilePage(site, page_name)
            
            try:
                page.upload(output_file_path, ignore_warnings=True, comment=summary)
            except Exception as e:
                print(e)
    site.logout()


# REQUIRED INFO FOR LOGIN AND FILE PATH

In [7]:
# Provide the necessary information here
file_path = './PDFs'  # Path to the folder containing PDF files
wiki_url = 'http://localhost'  # URL of your local MediaWiki server
username = 'YOUR USERNAME'  # Your MediaWiki username
# PASSWORD IS ASKED LATER...

In [21]:
# summary_buffer = "TESTINF SUMMARY. SOme random text for summary."
# add_summ = True

summary_buffer = ''
add_summ = False


# UPLOAD

In [ ]:

upload_pdf_files(file_path, wiki_url, username, addSummary = add_summ, summary = summary_buffer)